# Camino de santiago
* date : April 4 2017. ~ May 9 2017
* St. Jean Pied de Port ~ Santiago de Compostela.
* Santiago de Compostela ~ Fisterra

In [12]:
import os
from glob import glob
import sys
import pickle
from datetime import datetime



from pyicloud import PyiCloudService

from PIL import Image
from PIL.ExifTags import TAGS

import pandas as pd

import numpy as np

import folium
import gmaps
import googlemaps

from IPython.display import HTML, display

from ipywidgets.embed import embed_minimal_html

import ipywidgets as widgets

import base64
import matplotlib.pyplot as plt

%matplotlib

Using matplotlib backend: MacOSX


In [2]:
googleMapsKeys = pickle.load(open('./googleMapsKey','rb'))
#icloudLogin = pickle.load(open('./icloudLogInInformation','rb'))

In [3]:
def GetGPSInfo(file):
    from PIL import Image
    from PIL.ExifTags import TAGS
    try:
        img = Image.open(file)
        info = img._getexif()
        exif = {}
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            exif[decoded] = value
        exifGPS = exif['GPSInfo']
        exifDate = exif['DateTime']
        exifDate = datetime.strptime(exifDate, '%Y:%m:%d %H:%M:%S')
        exifDate = exifDate.date().strftime('%Y.%m.%d')
        latData = exifGPS[2]
        lonData = exifGPS[4]
        latDeg = latData[0][0] / float(latData[0][1])
        latMin = latData[1][0] / float(latData[1][1])
        latSec = latData[2][0] / float(latData[2][1])
        lonDeg = lonData[0][0] / float(lonData[0][1])
        lonMin = lonData[1][0] / float(lonData[1][1])
        lonSec = lonData[2][0] / float(lonData[2][1])
        # correct the lat/lon based on N/E/W/S
        Lat = (latDeg + (latMin + latSec / 60.0) / 60.0)
        if exifGPS[1] == 'S': 
            Lat = Lat * -1
        Lon = (lonDeg + (lonMin + lonSec / 60.0) / 60.0)
        if exifGPS[3] == 'W': 
            Lon = Lon * -1
        return Lat, Lon, exifDate
    except:
        pass

In [4]:
picturePath = '/Volumes/disk1/Pictures/'
fileList = glob(picturePath + '*.jpg')
print (len(fileList))

9462


In [5]:
gpsFromPicture = list(map(lambda x: GetGPSInfo(x), fileList))
gpsFromPicture = list(filter(None, gpsFromPicture))
gpsFromPicture = pd.DataFrame.from_records(gpsFromPicture, columns=['latitude', 'longitude', 'label'])
gpsFromPicture.dropna(inplace = True)
print (gpsFromPicture.shape)

(9268, 3)


In [6]:
gpsFromPicture.head()

,latitude,longitude,label
0,46.984464,8.253992,2017.06.13
1,36.719436,-4.423236,2017.05.28
2,42.952386,-9.189603,2017.05.07
3,42.340408,-3.703394,2017.04.18
4,41.379075,2.174117,2017.06.05


In [9]:
meanLat, meanLon = gpsFromPicture.latitude.mean(), gpsFromPicture.longitude.mean()

In [ ]:
travelmap = folium.Map(location=[meanLat, meanLon], zoom_start=10,
                    tiles='OpenStreetMap', control_scale = True)
for i in gpsFromPicture.index:
    if i % 2000 == 0: print (i)
    data = gpsFromPicture.loc[i]
    location = [data['latitude'], data['longitude']]
    name = data['label']
    marker = folium.Marker(location = location, popup = name)
    travelmap.add_child(marker)
travelmap.save('./test.html')
travelmap

0
2000
4000
6000
8000


In [11]:
# gmaps
gmaps.configure(api_key = googleMapsKeys)
mytravelPoints = gpsFromPicture.loc[:,['latitude', 'longitude']]
mytravel = gmaps.symbol_layer(mytravelPoints, fill_color='blue', stroke_color = 'blue', scale = 1)
fig = gmaps.figure()
fig.add_layer(mytravel)
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
embed_minimal_html('export.html', views=[fig])